In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/tisheedesh/Project/main/data.csv')

<ipython-input-3-e59cbfc6f7a6>:1: DtypeWarning: Columns (118) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('https://raw.githubusercontent.com/tisheedesh/Project/main/data.csv')


In [4]:
data

,Unnamed: 0,Артикул,Бренд,Для кого,Примерный вес,Тип металла,Проба,Покрытие,Тип вставки,Форма вставки,...,Чистота 6,Цветность 7,Чистота 7,Огранка 5,Цветность 8,Чистота 8,Цветность 9,Чистота 9,Цветность 10,Чистота 10
0,0,1012159-3,SOKOLOV,Для женщин,1.66 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1011493,SOKOLOV,Для женщин,1.35 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1012584-3,SOKOLOV,Для женщин,0.9 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1011116,SOKOLOV,Для женщин,2.28 г,Белое золото,585.0,NaN,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1011545-3,SOKOLOV,Для женщин,1.31 г,Белое золото,585.0,Родирование,Бриллиант (природный),Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10211,10211,94051161,SOKOLOV,Для женщин,4.1 г,Родированное серебро,925.0,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10212,10212,95010021,SOKOLOV,Для мужчин,12.34 г,Чернёное серебро,925.0,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10213,10213,95010022,SOKOLOV,Для мужчин,12.37 г,Чернёное серебро,925.0,NaN,Фианит,Круг,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10214,10214,95010034,SOKOLOV,Для мужчин,9.12 г,Чернёное серебро,925.0,Чернение,Без вставок,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.isna().sum()

Unnamed: 0           0
Артикул              0
Бренд                0
Для кого            27
Примерный вес        9
                 ...  
Чистота 8        10214
Цветность 9      10215
Чистота 9        10215
Цветность 10     10215
Чистота 10       10215
Length: 125, dtype: int64